In [22]:
# Copyright 2023 Shane Khalid. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

### SP500 Web Scraper

In [1]:
import numpy as np # linear algebra
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable


#### Use Webscraper to extract SP500 list from wikipedia, and using beautiful soup to parse and read

In [2]:
html_data=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text

In [3]:
beautiful_soup=BeautifulSoup(html_data, "html.parser")

In [4]:
tables = beautiful_soup.find_all('table')
len(tables)

2

In [14]:
S_P_500_companies = pd.DataFrame(columns=["Symbol","Security","Sector","Sub-Industry","Headquarters Location","Date first added","CIK","Founded"])

for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Symbol = col[0].text.strip().replace('\n','')
        Security = col[1].text.strip().replace('\n','')
        Sector = col[2].text.strip().replace('\n','')
        Sub_Industry = col[3].text.strip().replace('\n','')
        Headquarters_Location = col[4].text.strip().replace('\n','')
        Date_first = col[5].text.strip().replace('\n','')
        CIK = col[6].text.strip().replace('\n','')
        Founded = col[7].text.strip().replace('\n','')
        S_P_500_companies = S_P_500_companies._append({"Symbol":Symbol, "Security":Security, "Sector":Sector, "Sub-Industry":Sub_Industry,
        "Headquarters Location":Headquarters_Location,"Date first added":Date_first,"CIK":CIK,"Founded":Founded}, ignore_index=True)

#fixed append with _append

In [15]:
S_P_500_companies.head()

,Symbol,Security,Sector,Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [16]:
Symbols=S_P_500_companies['Symbol'].to_list()
len(Symbols)

503

#### Now use webscraper to extract SP500's componens

In [17]:
Symbols_data= {} # empty dictionary
for Symbol in Symbols:
    ticker_object = yf.Ticker(Symbol)

    #convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
    temp.reset_index(inplace=True)
    temp.columns = ["Attribute", "Recent"]
    temp['Symbol']=Symbol
    # add (ticker, dataframe) to main dictionary
    Symbols_data[Symbol] = temp
combined_data = pd.concat(Symbols_data)
combined_data

Attribute         Recent Symbol
MMM 0             address1      3M Center    MMM
    1                 city     Saint Paul    MMM
    2                state             MN    MMM
    3                  zip     55144-1000    MMM
    4              country  United States    MMM
...                    ...            ...    ...
ZTS 125       grossMargins        0.70238    ZTS
    126      ebitdaMargins        0.41219    ZTS
    127   operatingMargins        0.38257    ZTS
    128  financialCurrency            USD    ZTS
    129   trailingPegRatio         2.6403    ZTS

[64834 rows x 3 columns]

In [18]:
s=combined_data.pivot(index='Symbol', columns='Attribute', values='Recent')
s.reset_index(inplace=True)
s.columns.name=None
SP500Components=s.set_index('Symbol')
SP500Components.head()

,52WeekChange,SandP52WeekChange,address1,address2,ask,askSize,auditRisk,averageDailyVolume10Day,averageVolume,averageVolume10days,...,trailingAnnualDividendYield,trailingEps,trailingPE,trailingPegRatio,twoHundredDayAverage,underlyingSymbol,uuid,volume,website,zip
Symbol,,,,,,,,,,,,,,,,,,,,,
A,-0.257174,0.063376,5301 Stevens Creek Boulevard,NaN,105.72,800,10,2195090,1954093,2195090,...,0.008484,3.82,26.903141,2.4063,128.1895,A,7fc56270-e7a7-3fa8-9a59-35b72eacbe29,2004018,https://www.agilent.com,95051
AAL,-0.229901,0.063376,1 Skyview Drive,NaN,10.93,1100,6,45674580,30489679,45674580,...,0.0,2.39,4.569037,0.0378,14.9568,AAL,b73d98bf-8d5e-382f-9d49-b84d8f52f152,39890723,https://www.aa.com,76155
AAPL,0.097039,0.063376,One Apple Park Way,NaN,168.03,800,4,57437060,58993159,57437060,...,0.005573,5.96,28.224833,2.1778,170.53055,AAPL,8b10e4ae-9eeb-3684-921a-9ab27e4d87aa,58499129,https://www.apple.com,95014
ABBV,-0.051025,0.063376,1 North Waukegan Road,NaN,139.06,1200,9,5208220,4300440,5208220,...,0.040771,4.86,28.586418,5.4642,148.12555,ABBV,73238d42-cdcc-3f92-8141-dd675addae10,9427212,https://www.abbvie.com,60064-6400
ABNB,0.067159,0.063376,888 Brannan Street,NaN,114.3,1100,6,4480280,6559317,4480280,...,0.0,3.41,33.457478,1.2032,124.44643,ABNB,403113aa-7838-3345-9d32-d63a5f9da70b,5201552,https://www.airbnb.com,94103


In [21]:
import os
os.chdir('./')
SP500Components.to_csv('./S&P500_Components.csv')
S_P_500_companies.to_csv('./S&P500_Companies.csv')